In [2]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

### Introduction to Finance - Weighted Average Cost of Capital

Welcome back Kagglers! We are continuing our two-part series of discounted cash flow models (DCF) to calculate the value of a stock. In the last tutorial, we calculated Caterpillar's (CAT) stock price based on the Dividend Discount Model (DDM). We discovered that the dividend is a poor proxy for cash flow, so we are moving on to a more sophisticated DCF model called the weighted average cost of capital (WACC) model.

We will cover the following information in this tutorial:

  +  Statement of Cash Flows Overview
  +  Company Valuation Overview
  +  Free Cash Flow
  +  Company Valuation Example
  +  Python Script and Tips


### Statement of Cash Flows Overview
We will continue to evaluate CAT's recent financial statements and build off our learnings of the Income Statement (IS) and Balance Sheet (BS).

In the last tutorial, we learned that the IS is a summary of a business's performance over a time period and is measured in terms of Net Income (Revenue - Expenses). Additionally, we learned that the BS is a snapshot in time and measures the company's health in terms assets, liabilities, and shareholder equity.

During this section we will introduce a new financial statement called the Statement of Cash Flows (SCF).

**Statement of Cash Flows**: The SCF calculates how changes in the IS and BS affect cash flow. Cash flow is critical to understand because it is the most common reason that companies go bankrupt. The SCF categorizes cash into operating, investing, and financing activities. As a general rule of thumb, businesses aim to convert net income/operating profit to cash on a 1 to 1 basis. We can dig more into this later. In personal finance, you can analyze your cash flow to see how much money you can invest after your everyday expenses are deducted from your income. The higher, the better.



### Company Valuation Overview (WACC)

The WACC model expands on the DDM's assumption that a company's value is based on cash flow.  The DDM assumes that the dividend is a good proxy for cash flow.  However, we discovered that the drawbacks of valuation by dividend were: 

    1) A company that doesn't pay a dividend can't be valued
    
    2) The dividend is not always a good proxy of cash flow because it doesn't contain perfect information.  
    
The WACC model digs deeper by calculating cash flow from the company financial statements.  


Next, let's jump into the WACC calculation.  We will assume a constant growth rate of 6% for simplicity of implementing this model.  Six percent is a common proxy used as a starting point for growth rates (1).  There are several components to this calculation, so this breakout is more in-depth than the DDM.

**WACC Valuation Assumption:**
  +  Calculation: Value = Debt + Equity
  +  Assumption: Constant Growth @ 6%
  +  Assumption: Ignore After-Tax Interest for simplicity


The BS contains the name 'balance' because assets must equal debt + equity.  Therefore, the WACC calculation makes sense that a company is worth its debt (how much it borrowed) plus its equity (how much stock it has issued).  A company receives cash when it borrows or issues stock, which in turn it invests into the business to turn a profit. 

Nothing is ever free, so the company must pay the bondholder and stockholders a return for their investment.  This return is called the cost of capital or in laymen terms, the cost of raising money to run a business.

**WACC Return Assumption:**
  +  Calculation: RWACC = (1–tc)·xD·RD + xE ·RE
    + RD = Return on Debt
    + XD = Proportion of Debt
    + tc = Corporate Tax Rate
    + RE = Return on Equity
    + XE = Propotion of Equity

This formula looks intimidating, but as we explore each variable, you will see it's quite straightforward.  Let's start with debt and equity which represent the total investment in the company.  For debt we use long term debt which can be found on the BS.  Additionally, for equity we use the number of shares outstanding times the current share price.  xD and xE are the respective weights of debt and equity in terms of total investment in the company.

Next, it is common to assume that the Corporate Tax Rate (tc) is equal to 35% for simplicity (1).  Finally, we were given RE in the DDM example, and we know it is equal to 9.445%.  We are just missing one piece of information, which is the return on debt.  This information is readily available in the annual report and is 5.5%.  Whew, that wasn't so bad.  Now it's just math to put the calculation together.

### Company Valuation - WACC Pulling It All Together
We now understand how the WACC model will evaluate the return on debt and equity. Additionally, we understand that the WACC model assumes a company's value is based on debt and equity. What are we missing then? Well, since the WACC model is a discounted cash flow valuation method, we are missing... cash flow!

Therefore, we can remove debt and equity from the calculation and re-define the company value as:
V = All Future Free Cash Flows

Remember from the DDM model that we defined future cash flows as the dividend. In the WACC model, we will take this a level deeper by accounting for cash flows that can be received by debt holders and shareholders. More specifically, we will review the SCF to determine how much cash a company is generating.

### Free Cash Flow Overview

Free cash flow (FCF) is defined as follows:

**FCF Assumptions:**
  +  Calculation: FCF = Net Income - Capital Expenditures + Depreciation - Increase in Working Capital

**Net Income** is the amount of profit or loss a company earns during a particular time period.  It is the starting point of the FCF calculation because it represents the cash available after revenue and expenses have all been accounted.  It's no different than balancing your checkbook to see how much cash you have available to utilize for the future.  -

** Capital Expenditures** are assets purchased over a certain dollar threshold such as \$2500.  These assets are paid for with cash raised from debt, equity, or net income by a company.  For example, uses \$200,000 to buy a new machine, then this is a cash outflow.

**Depreciation** is a term used to describe the loss in value ('wear and tear') of an asset.  For anyone who has purchased a car, you know that your car depreciates as your mileage increases.  Depreciation is a non-cash expense because the asset is paid for as part of the capital expenditures discussed above.  Depreciation reduces the value of an asset every year by expensing the value loss to the IS.  Depreciation results in tax savings called a depreciation tax shield that allows company to reduce their income and pay fewer taxes for the perceived loss of an asset (2).  Therefore since depreciation is a non-cash expense, we add it back to FCF.


**Working Capital** is defined as a company's current assets less its current liabilities.  'Current' means assets or liabilities readily convertible to cash in the next 12 months (2).  Asset examples include Accounts Receivable and Inventory.  Liability examples include Accounts Payable and Expense Accruals.  An *increase* of an asset is a **use** of cash, and a *increase* of a liability is a **source** of cash.  Think about it this way, Accounts Receivable (AR) is money owed from a customer to a company.  Therefore, AR is a short-term loan that is provided to a customer for a set term (i.e., 30 days) and is a reduction in cash for a company until the customer pays.  Additionally, inventory cost the company cash to build it and ties up cash until it is converted until a sale and the accounts receivable is collected.

### Company Valuation Example

The components of the FCF equation reside on several financial statements, and we will define the components as we add them to a database.  Now let's roll up those sleeves because it's time to do some data wrangling to calculate FCF.

Let's start by adding in the IS and BS. We will then move onto the SCF.

In [185]:
bs_df = pd.read_csv("CAT_BS.csv")
bs_df.set_index("Accounts", inplace=True)

In [92]:
bs_df

,Category,TY2016,TY2015
Accounts,,,
Cash and short-term investments,Assets,7168,6460
Receivables - trade and other,Assets,5981,6695
Receivables - finance,Assets,8522,8991
Prepaid expenses and other current assets,Assets,1682,1662
Inventories,Assets,8614,9700
"Property, plant and equipment - net",Assets,15322,16090
Long-term receivables - trade and other,Assets,1029,1170
Long-term receivables - finance,Assets,13556,13651
Noncurrent deferred and refundable income taxes,Assets,2790,2489


In [6]:
bs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 4 columns):
Accounts    31 non-null object
Category    31 non-null object
TY2016      31 non-null int64
TY2015      31 non-null int64
dtypes: int64(2), object(2)
memory usage: 1.0+ KB


**Excel tip**: 'Groupby' in Python gives a summary of the data that is similar to Excel's pivot table. I have attached the '.sum()' method to the groupby command to provide the sum for each category of the BS.


In [7]:
bs_df.groupby(['Category'])['TY2016'].sum()

Category
Assets                74704
Liability             61491
Stockholder Equity    13213
Name: TY2016, dtype: int64

In [97]:
is_df = pd.read_csv("CAT_IS.csv")
is_df.set_index("Account", inplace=True)

In [98]:
is_df

,TY2016,TY2015,TY2014
Account,,,
"Sales of Machinery, Energy & Transportation",35773.00,44147.00,52142.0
Revenues of Financial Products,2764.00,2864.00,3042.0
Total sales and revenues,38537.00,47011.00,55184.0
Cost of goods sold,28309.00,33546.00,40718.0
"Selling, general and administrative expenses",4686.00,4951.00,6529.0
Research and development expenses,1951.00,2119.00,2380.0
Interest expense of Financial Products,596.00,587.00,624.0
Goodwill impairment charge,595.00,0.00,0.0
Other operating (income) expenses,1902.00,2023.00,1619.0


Now we will add the SCF.  Remember it describes elements from the BS and IS.

In [11]:
cf_df = pd.read_csv("CAT_CF.csv")

In [12]:
cf_df

,Account,Category,TY2016,TY2015,TY2014
0,Net Income,Operating Activity,-59,2523,2468
1,Depreciation and amortization,Operating Activity,3034,3046,3163
2,Actuarial (gain) loss on pension and postretir...,Operating Activity,985,179,2624
3,Provision (benefit) for deferred income taxes,Operating Activity,-431,-307,-954
4,Goodwill impairment charge,Operating Activity,595,0,0
5,Other,Operating Activity,856,453,470
6,Receivables - trade and other,Operating Activity,829,764,163
7,Inventories,Operating Activity,1109,2274,101
8,Accounts payable,Operating Activity,-200,-1165,222
9,Accrued expenses,Operating Activity,-201,-199,-10


In [13]:

cf_df.groupby(['Category'])['TY2016','TY2015', 'TY2014'].sum()

,TY2016,TY2015,TY2014
Category,,,
FX,-28,-169,-174
Financing Activity,-6224,-7740,-5992
Investing Activity,-1760,-3517,-3627
Operating Activity,5608,6675,8057
Total,7168,6460,7341


### Company Valuation Example: FCF Calculation

We have now added the three financial statements into databases in Python. Next, we will estimate CAT's free cash flow by selecting information from the SCF.

As discussed earlier, Free Cash Flow (FCF) is calculated by the following formula:

#### FCF = Net Income + Depreciation - Capital Expenditures - Increase in Working Capital

First, we will start by looking at Net Income on the IS. I remember that Net Income was at the bottom of the IS, so let's pull the last few rows to find it. The '.tail()' method in Python allows you to pull the last five rows of a database. After using this method, I see that net income is row 17 of the IS.

In [14]:
is_df.tail()

,Account,TY2016,TY2015,TY2014
15,Profit (loss) of consolidated companies,-53.00,2523.00,2460.0
16,Equity in profit (loss) of unconsolidated affi...,-6.00,0.00,8.0
17,Net Income,-59.00,2523.00,2468.0
18,Basic Shares Outstanding (millions),584.30,594.30,617.2
19,Dividend per Share,3.08,3.01,2.7


I remember seeing Net Income also on the Cash Flow Statement, so let me pull it to see if it matches Net Income on the IS.

In [15]:
cf_df[cf_df.Account == 'Net Income']

,Account,Category,TY2016,TY2015,TY2014
0,Net Income,Operating Activity,-59,2523,2468


Perfect, we have demonstrated that Net Income from the IS is the starting point (i.e. first row) of the SCF.  Depreciation is also on the IS and included on the SCF, but our IS is not detailed enough to show depreciation.  Therefore, we will just assume the Depreciation value on the SCF is accurate.

FCF = **Net Income + Depreciation** - Capital Expenditures - Increase in Working Capital

We have identified where the first two components of FCF reside, so now let's move to Capital Expenditures (CAP_EX) and Working Capital.

CAP_EX and Working Capital both reside on the BS.  However, our BS is detailed enough to show CAP_EX, so we will assume the CAP_EX as given on the SCF.  Working Capital is made up of rows 6 - 10 (Receivables, Inventory, Accounts Payable, and Accrued Costs) on the SCF.  Let's take a look.

In [16]:
cf_df[6:11]

,Account,Category,TY2016,TY2015,TY2014
6,Receivables - trade and other,Operating Activity,829,764,163
7,Inventories,Operating Activity,1109,2274,101
8,Accounts payable,Operating Activity,-200,-1165,222
9,Accrued expenses,Operating Activity,-201,-199,-10
10,"Accrued wages, salaries and employee benefits",Operating Activity,-708,-389,901


The action of identifying a subset of a database is called 'slicing' in Python. In this example, we slice the dataframe by identifying the rows that we are interested in on the SCF.

Inventory is showing a \$1,109 *source* of cash for 2016.  Let's see if we can tie this *decrease* in inventory to the BS.

In [17]:
bs_df['2016_CF'] = bs_df['TY2016']-bs_df['TY2015']

In [18]:
bs_df[4:5]

,Accounts,Category,TY2016,TY2015,2016_CF
4,Inventories,Assets,8614,9700,-1086


Oh No! The BS is showing a reduction in inventory of \$1,086, but the SCF is showing a reduction of \$1,109. Did we do something wrong? Nope, it's just accounting. Not all accounting entires have a cash flow impact.

**ProTip:** Companies adjust the SCF for accounting related non-cash entries. For example, non-cash accounting entries such as obsolete invetory, slow-moving inventory, and impaired inventory can be removed when calculating cash flow. Cash flow adjustments apply to all cash flows categories. For example, accounting entries such as the Bad Debt Reserve are removed from AR to calculate cash flow (3). Therefore, you will not be able to always tie out movements on the BS to the SCF.

Okay, that's great, but how do I take row 17 and add it to another database. This action of removing a row from a database is called 'Slicing' in Python. There are many ways to slice a database, but the safest way is to call out the name of the item that you want to move. For example, the code below first states that I want access the IS dataframe 'is_df[' and then once I am in that data frame, I want to look into the description column and access the row that is called 'Net Income'.

**Excel Tip:** This slicing example is similar to doing a vlookup for a criteria in Excel.

Ok, let's create a dataframe to calculate FCF by starting with Net Income.

In [77]:
FCF_df = cf_df[cf_df.Account == 'Net Income']

Let's now add depreciation to our FCF_df using the same code logic that we added Net Income.  Notice we are using the 'append' Python method to add a new row to our dataframe.

**Python Tip**: Be careful when using the 'append' method because it will continue to append duplicate rows every time you execute the line of code below. 

In [78]:
FCF_df= FCF_df.append(cf_df[cf_df.Account == 'Depreciation and amortization'])

Another way to slice data in a dataframe is by selecting the rows that you want.  Python starts with row zero, so be mindful of this when slicing data by numbers.  We will append the Capital Expenditures by selecting row 14.

In [79]:
FCF_df = FCF_df.append(cf_df[14:15])

Next lets now append rows 6 thru 10 to include working capital in our FCF database.

In [80]:
FCF_df = FCF_df.append(cf_df[6:11])

Perfect, we now have a database that we can use to calculate FCF.  Let's add Totals to the columns so that we can see the FCF by year.

In [81]:
FCF_df.loc['Total']= FCF_df.sum(numeric_only=True)

In [82]:
FCF_df

,Account,Category,TY2016,TY2015,TY2014
0,Net Income,Operating Activity,-59.0,2523.0,2468.0
1,Depreciation and amortization,Operating Activity,3034.0,3046.0,3163.0
14,Capital expenditures - excluding equipment lea...,Investing Activity,-1109.0,-1388.0,-1539.0
6,Receivables - trade and other,Operating Activity,829.0,764.0,163.0
7,Inventories,Operating Activity,1109.0,2274.0,101.0
8,Accounts payable,Operating Activity,-200.0,-1165.0,222.0
9,Accrued expenses,Operating Activity,-201.0,-199.0,-10.0
10,"Accrued wages, salaries and employee benefits",Operating Activity,-708.0,-389.0,901.0
Total,NaN,NaN,2695.0,5466.0,5469.0


The WACC calculation will be based on the 2016 FCF, so we will store that into a variable called FCF_2016 for later use.

In [83]:
FCF_2016 = FCF_df.loc["Total","TY2016"]

In [84]:
FCF_2016 

2695.0

### Company Valuation Example: WACC Calculation


Next we need to jump into the WACC calculation and assign values to each component.

Calc: WACC = (1–tc)·xD·RD + xE ·RE
RD = Return on Debt
XD = Proportion of Debt
tc = Corporate Tax Rate
RE = Return on Equity
XE = Propotion of Equity

We will start with weights of debt (xD) and equity (xE).  For debt, we use Long-Term Debt, which can be found on the BS.  Additionally, equity is calculated by multiplying the number of shares outstanding times the current stock price. 

In [160]:
Long_Term_Debt = bs_df.loc["Long Term Debt","TY2016"]
Long_Term_Debt

14382

Shares outstanding can be found on the IS.  We will create a variable for the 2016 shares outstanding.

In [99]:
Shares_Outstanding = is_df.loc["Basic Shares Outstanding (millions)","TY2016"]

Next, let's get CAT's stock price.  We can pull a year's worth of stock prices for CAT from Google Finance and create a dataframe called price_df.  We can do this using pandas datareader.

In [137]:
import pandas_datareader.data as web
import pandas_datareader.data as getData
import datetime

In [151]:
price_df = getData.get_data_google("CAT")

In [153]:
price_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 250 entries, 2016-11-30 to 2017-11-27
Data columns (total 5 columns):
Open      249 non-null float64
High      249 non-null float64
Low       249 non-null float64
Close     250 non-null float64
Volume    250 non-null int64
dtypes: float64(4), int64(1)
memory usage: 11.7 KB


In [154]:
price_df

,Open,High,Low,Close,Volume
Date,,,,,
2016-11-30,95.75,95.90,94.35,95.56,4641863
2016-12-01,96.00,97.39,94.05,96.24,9361618
2016-12-02,95.36,96.15,94.93,95.14,5038257
2016-12-05,95.87,95.99,94.15,94.45,4980335
2016-12-06,94.10,95.31,94.07,95.22,3663143
2016-12-07,95.25,97.33,94.93,97.33,4901054
2016-12-08,97.26,97.40,96.13,96.27,3861626
2016-12-09,96.23,96.30,95.19,95.53,6550220
2016-12-12,96.01,96.65,94.91,95.08,3597051


In tutorial 1, we used the DDM model based on the share price of November 20, 2017.  Let's also use that share price for the WACC model.

In [157]:
share_price = price_df.loc["2017-11-20","Close"]
share_price

136.91

In [158]:
Equity = share_price * Shares_Outstanding
Equity

79996.512999999992

In [161]:
Debt = int(Long_Term_Debt)
Equity = int(Equity)
Total = Debt + Equity
Total

94378

In [162]:
xD =Debt / Total
xD

0.15238720888342622

In [163]:
xE =Equity / Total
xE

0.8476127911165737

After running the calculations, we can see that CAT's capital structure is made up of 85% equity and 15% debt.

Next, we will use the return on equity (ROE), that was given in tutorial 1.

In [164]:
Beta = 1.29 #(3) Need 5 year avg
Market_Risk_Prem =0.055 #(1) Chapter 7
RF = 0.0235 #(5) 10 year treasury rate

RE = RF + Market_Risk_Prem * Beta
print (RE)

0.09445


Now we assign the tax rate variable.

In [177]:
tc = 0.35

In [186]:
# This is an estimate of CATs long term debt section on their annual report.
RD = 0.055

In [187]:
WACC = (1-tc)*xD*RD + xE*RE

In [188]:
WACC

0.08550487083854288

### Company Valuation Example: Pulling it All Together

In [189]:
# We define the growth rate of 6% to use in the valuation calc.
g = 0.06

In [190]:
Vtwacc = (FCF_2016*(1-g)) / (WACC - g)

The WACC calculation yields the total valuation of the company including debt.  To calculate the stock price, we need to subtract the value of long term debt and then divide the equity value by the shares outstanding.

In [191]:
Vewacc= Vtwacc - Debt

In [192]:
Vewacc / Shares_Outstanding

145.37758930444252

The WACC model estimates the share price at \$145, which is higher than the actual price of \$137.  Therefore, the WACC model believes CAT is undervalued.  However, please note that this WACC calculation assumes constant growth of 6% and that valuations can fluctuate widely with changes to the growth rate (g).  More advanced WACC calculations can factor in multiperiod growth rates, which could perhaps lead to a more accurate stock price prediction.  

The goal of this tutorial was to provide a foundation for calculating free cash flow and using it to derive a company stock price.  We showed how FCF was calculated and used to replace the dividend as a cash flow proxy. Please leave comments to let us know how we did!

Next up we will learn how to value a company based on ratios and relative benchmarks in the market and industry.

Works Cited:
1. The Financial Times Guide to Understanding Finance, 2nd Edition by Javier Estrada Published by Pearson Business, 2011
2. Fundamentals of Corporate Finance, 6th Edition by Ross, Westerfield, and Jordan Published by McGraw-Hill Irwin, 2003
3. http://www.accounting-basics-for-students.com/-bad-debts-in-cash-flow-statement-.html